In [2]:
import numpy as np
from numpy import zeros,ones,linalg
from sklearn.cross_validation import train_test_split
import mosek
import math
import sys


# Define a stream printer to grab output from MOSEK
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()


def kernel(x1,x2):
    return math.exp(-1*math.pow(linalg.norm(x1-x2),2)/(2*math.pow(sigma,2)))

sigma = 10

x = []
f = open('bunny.obj.txt')
for line in f:
    line = line.strip()
    if line != '' and line[0] != '#':
        line_split = line.split()
        if len(line_split)==4 and line_split[0] == 'v':
            x.append(line_split[1:])
x = np.array(x)
x=x.astype(np.float)
y = np.ones(len(x))
m = len(x)
print 'points:',m
x,ignore_x,y,ignore_y=train_test_split(x,y,train_size=0.1,random_state=8)
m = len(x)
print 'points:',m

# overfit/underfit param
v=.99

# Open MOSEK and create an environment and task
# Make a MOSEK environment
env = mosek.Env ()
# Attach a printer to the environment
env.set_Stream (mosek.streamtype.log, streamprinter)
# Create a task
task = env.Task()
task.set_Stream (mosek.streamtype.log, streamprinter)
# Set up and input bounds and linear coefficients
bkc   = [ mosek.boundkey.fx]
blc   = [ 1.0 ]
buc   = [ 1.0 ]

bkx=[]
for i in range(m):
    bkx.append(mosek.boundkey.ra)
bux=(ones((m))) * (1/(v*m))
blx=(ones((m))) * (-1/(v*m))
c=(zeros((m)))

numvar = len(bkx)
numcon = len(bkc)

# Append 'numcon' empty constraints.
# The constraints will initially have no bounds.
task.appendcons(numcon)

# Append 'numvar' variables.
# The variables will initially be fixed at zero (x=0).
task.appendvars(numvar)

for j in range(numvar):
    # Set the linear term c_j in the objective.
    task.putcj(j,c[j])
    # Set the bounds on variable j
    # blx[j] <= x_j <= bux[j]
    task.putbound(mosek.accmode.var,j,bkx[j],blx[j],bux[j])

    # Input i, j of A
    task.putaij(0, j, 1)

for i in range(numcon):
    task.putbound(mosek.accmode.con,i,bkc[i],blc[i],buc[i])

# Input the objective sense (minimize/maximize)
task.putobjsense(mosek.objsense.maximize)

# Set up and input quadratic objective
for i in range(m):
    for j in range(i+1):
        task.putqobjij(i,j,kernel(x[i],x[j]))

task.putobjsense(mosek.objsense.minimize)

# Optimize
task.optimize()
# Print a summary containing information
# about the solution for debugging purposes
task.solutionsummary(mosek.streamtype.msg)

prosta = task.getprosta(mosek.soltype.itr)
solsta = task.getsolsta(mosek.soltype.itr)

# Output a solution
xx = zeros(numvar, float)
task.getxx(mosek.soltype.itr, xx)

if solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal:
    print("Optimal solution: %s" % xx)
elif solsta == mosek.solsta.dual_infeas_cer:
    print("Primal or dual infeasibility.\n")
elif solsta == mosek.solsta.prim_infeas_cer:
    print("Primal or dual infeasibility.\n")
elif solsta == mosek.solsta.near_dual_infeas_cer:
    print("Primal or dual infeasibility.\n")
elif  solsta == mosek.solsta.near_prim_infeas_cer:
    print("Primal or dual infeasibility.\n")
elif mosek.solsta.unknown:
    print("Unknown solution status")
else:
    print("Other solution status")

alpha = xx

x_test = x[0] #test at x
w_x=0
for i in range(m):
    w_x = w_x + (alpha[i]*kernel(x[i],x_test))

print 'w = ',w_x


points: 2503
points: 250
Computer
  Platform               : MACOSX/64-X86   

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 1               
  Cones                  : 0               
  Scalar variables       : 250             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Eliminator - elim's                 : 0               
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Matrix reordering started.
Local matrix reordering star